Before running the code, several dependant package need to be installed, skip if it already exists.

In [2]:
pip install urllib3

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install pdfminer.six

     |████████████████████████████████| 5.6MB 5.3MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install python-Levenshtein

Firstly, we make connection with database with pymysql api.

In [1]:
import pymysql
#input basic info about database
conn=pymysql.connect(host='mysql24.ezhostingserver.com', port=3306, user='krw',password='Y>!V@N_26@]cfJ7(')
#cursor is the current point we focus on, which could execute sql command.
cursor=conn.cursor()
lst=[]
#Here krw is the databse name and training_view is a view for requesting data. The execution should be around 30s.
cursor.execute("SELECT * FROM krw.training_view limit 10;")
for r in cursor:
    lst.append(list(r))
lst

[['Low-shot learning with large-scale diffusion',
  4,
  'graph construction',
  'https://github.com/facebookresearch/low-shot-with-diffusion',
  '1706.02332',
  'YES'],
 ['A Neural Temporal Model for Human Motion Prediction',
  5,
  'motion prediction',
  'https://github.com/cr7anand/neural_temporal_models',
  '1809.03036',
  'YES'],
 ['Integrating kinematics and environment context into deep inverse reinforcement learning for predicting off-road vehicle trajectories',
  5,
  'motion prediction',
  'https://github.com/yfzhang/vehicle-motion-forecasting',
  '1810.07225',
  'YES'],
 ['Spectral Inference Networks: Unifying Deep and Spectral Learning',
  5,
  'bilevel optimization',
  'https://github.com/deepmind/spectral_inference_networks',
  '1806.02215',
  'YES'],
 ['Collaborative Motion Prediction via Neural Motion Message Passing',
  4,
  'motion prediction',
  'https://github.com/PhyllisH/NMMP',
  '2003.06594',
  'YES'],
 ['Common-Knowledge Concept Recognition for SEVA',
  4,
  'gr

After excution we obtained a 2-dimensional matrix with each row represents a paper. Next, we constract a url based on arXiv id.

In [136]:
name=lst[4][4]
url="https://arxiv.org/pdf/"+str(name)+".pdf"
print(url)


https://arxiv.org/pdf/2003.06594.pdf


By following command we could access pdf by url and convert pdf 2 txt file and store at local.

In [137]:
#https://cloud.tencent.com/developer/article/1395339
import urllib
import pdfminer
from io import StringIO
from io import BytesIO
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import *
from pdfminer.converter import PDFPageAggregator
from urllib.request import Request
from urllib.request import urlopen

def OnlinePdfToTxt(dataIo,new_path):
    # Create PDF Parser
    parser = PDFParser(dataIo)
    # Create PDFDocument
    document = PDFDocument(parser)
    # Is it okay for extraction?
    if not document.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        # Create PDF Manager
        resmag =PDFResourceManager()
        # Setting parameters
        laparams=LAParams()
        # Createing PDF device
        # device=PDFDevice(resmag )
        device=PDFPageAggregator(resmag ,laparams=laparams)
        # Create PDF explainer
        interpreter=PDFPageInterpreter(resmag ,device)
        # For each page
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            # accept this page's LTP object
            layout=device.get_result()
            for y in layout:
                try:
                    if(isinstance(y,LTTextBoxHorizontal)):
                        with open('%s'%(new_path),'a',encoding="utf-8") as f:
                            f.write(y.get_text()+'\n')
#                             print("Success！")
                except:
                    print("Failed")

#url = "https://arxiv.org/pdf/2004.11055.pdf"
html = urllib.request.urlopen(urllib.request.Request(url)).read()
dataIo = BytesIO(html)
OnlinePdfToTxt(dataIo,'txt/'+str(name)+'.txt')

In [138]:
def judge(x):
    for string in x:
        
    return bool(re.search(r'\d', x))

def author_extraction(file_name):
    file = open('txt/'+str(file_name)+'.txt',encoding="ISO-8859-1")
    strings=file.read()
    keyStart = ''
    keyEnd = 'Abstract'
    pat = re.compile(keyStart+'(.*?)'+keyEnd, re.S)
    result = pat.findall(strings)
#     if len(result)<10:
#         return
    #print(result)
    file.close()
    #format processing
    txt=''
    txt=txt.join(result)
    txt=txt.replace('\n', '. ').replace('\r', ' ')
    print(txt)
    rst=[]
    tempList=txt.split()
    for x in tempList:
        if not judge(x):
            rst.append(x)
        else:
            rst.append(' ')
    output=' '.join(rst)
    print(output)
    #using the pretrained model
    doc=nlp(output)
    ORG_list=[]
    #print([(X.text, X.label_) for X in doc.ents])
    for X in doc.ents:
        if X.label_=='PERSON':
            ORG_list.append(X.text)
    return ORG_list
author_extraction(name)

Collaborative Motion Prediction via Neural Motion Message Passing. . Yue Hu1, Siheng Chen2 (cid:0), Ya Zhang1 (cid:0), and Xiao Gu1. . 1 Cooperative Medianet Innovation Center, Shanghai Jiao Tong University. 2 Mitsubishi Electric Research Laboratories. {18671129361, ya zhang, gugu97} @sjtu.edu.cn, schen@merl.com. . 0. 2. 0. 2.  . r. a. . M.  . 4. 1.  .  . ]. . V. C. .. s. c. [.  .  . 1. v. 4. 9. 5. 6. 0. .. 3. 0. 0. 2. :. v. i. X. r. a. . 
Collaborative Motion Prediction via Neural Motion Message Passing. . Yue   Siheng     Ya     and Xiao   .   Cooperative Medianet Innovation Center, Shanghai Jiao Tong University.   Mitsubishi Electric Research Laboratories.   ya zhang,   @sjtu.edu.cn, schen@merl.com. .         . r. a. . M. .     . . ]. . V. C. .. s. c. [. . .   v.           ..         :. v. i. X. r. a. .


['Yue   Siheng', 'Xiao   ', 'V. C.']

In [126]:
L = ['xiaohong', '12', 'adf12', '14..', '晓明']


judge("...12.")
# 对于python3来说同样还可以使用string.isnumeric()方法
# for x in L:
#     if not x.isdigit():
#         print(x)

True

From the generated txt file, first we try to extract the email address by using regular experssion
### TODO: Special format such as {aaa, bbb, ccc}@xxx.com


In [50]:
import re



#open the generated file as input stream and store in the buffer
def email_extraction(file_name):
    file = open('txt/'+str(file_name)+'.txt',encoding="ISO-8859-1")
    strings=file.read()
    matches = []
    emailRegex = re.compile(r'''(
        [a-zA-Z0-9._%+-]+      # username

        @                      # @ symbol

        [a-zA-Z0-9.-]+        # domain name

        (\.[a-zA-Z]{2,4}){1,2} # dot-something

        )''', re.VERBOSE)
    # using RE to match all the patterns in the txt
    for groups in emailRegex.findall(strings):
        matches.append(groups[0])
    # reduced the same entities
    list2 = list(set(matches))
    list_nums = len(list2)
    emailList=[]
    for line in range(list_nums):
        emailList.append(list2[line])
    return emailList

email_extraction(name)

['nn@eecs.berkeley.edu']

Secondly we match the author name list with the email list by token matching method (Levenshtein)
### TODO: HOW can we access author list? 

In [6]:
from Levenshtein import *
def author_email_matching(nameList, emailList):
    if len(emailList)==0:
        return
    name_email={}
    for name in nameList:
        highest=0
        index=0
        for i in range(len(emailList)):
            #using re to extract the first part of email
            pat = re.compile(''+'(.*?)'+'@', re.S)
            email = pat.findall(emailList[i])
            #print(jaro(name, str(email)))
            #using jaro to calculate the similarity between two strings
            if jaro(name, emailList[i])>highest:
                index=i
                highest=jaro(name, emailList[i])
        #set pair with the highest score
        name_email[name]=emailList[index]
        #If very sure, remove it from the list to reduce the uncertainty for other pairs
        if highest>=0.6:
            emailList.remove(emailList[index])
    return name_email
        

nameList=['Michael Stewart', 'Majigsuren Enkhsaikhan', 'Wei Liu']
emailList=email_extraction(name)
author_email_matching(nameList, emailList)

{'Michael Stewart': 'michael.stewart@research.uwa.edu.au',
 'Majigsuren Enkhsaikhan': 'majigsuren.enkhsaikhan@research.uwa.edu.au',
 'Wei Liu': 'wei.liu@uwa.edu.au'}

After matching email, we move to the section of acknowledgement, which is not certain for every paper. Here we use named entity recognition to classify orgnization which sponsred the study. Here we used Spacy classifier.

In [7]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [8]:
#using Regular expression to locate the paragraph
def ORG_recognition(strings):
    keyStart = 'Acknowledg+'
    keyEnd = 'References'
    pat = re.compile(keyStart+'(.*?)'+keyEnd, re.S)
    result = pat.findall(strings)
    if len(result)<10:
        return
    #print(result)
    file.close()
    #format processing
    txt=''
    txt=txt.join(result)
    txt=txt.replace('\n', '').replace('\r', '')
    #print(txt)
    #using the pretrained model
    doc=nlp(txt)
    ORG_list=[]
    print([(X.text, X.label_) for X in doc.ents])
    for X in doc.ents:
        if X.label_=='ORG':
            ORG_list.append(X.text)
    return ORG_list
ORG_recognition(0)